# 데이터 증식을 이용하여 이미지 분류하기

## 모델 생성의 두번 째 방법

In [1]:
### 필요한 라이브러리 임폴트
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random

In [2]:
'''
모델 생성의 첫번 쨰 방법
model = tf.keras.Sequental()
model.add(tf.keras.layer.Conv2d()
'''

'''
### 모델 생성의 두번 째 방법
model = tf.keras.Model(input, output)
'''
# 인풋에 널고 아웃풋 결과만 널어 모델을 만드는 구조만 만들어두면 뚝딱 만들어짐
# 배열 만들고 어떤 함수에 입력으로 주고 그 결과 값을 새로운 함수 입력에 넣으면 또 결과값이 나오는데 이걸 반복
# 딥러닝 컵퓨터 비전 p.48 맨위에것 써서 p.51 모델을 만들음 (강아지그림)

# 입력 생성용 전용 합수
inputs = tf.keras.Input(shape=(3,))

# 출력 생성
outputs = tf.keras.layers.Dense(units=4, activation='softmax')(inputs)

# 모델 생성
model = tf.keras.Model(inputs, outputs)

# 결과 확인
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 4)                 16        
                                                                 
Total params: 16 (64.00 Byte)
Trainable params: 16 (64.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 모델 생성

In [3]:
### 기본 모델 생성하기
base_model = tf.keras.applications.vgg16.VGG16(weights=None)

# 모델 구조 확인
base_model.summary()

# TensorFlow 에서 제공하는 여러 모델들 중 vgg16이라는 모델에서 vgg16이라는 파이썬 파일이 있음 (okk.OKK가 있듯)
# Keras / keras / applications / vgg16
# 그냥 불러오면 있는 것들 다 가져오니까 weight (지정할게 없거나, 사전학습된 모델을 넣고 싶으면 가중치를 다 가지고 있는 모델을 가져옴)


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0    

In [4]:
### 학습 전 기본 모델의 가중치 확인
print(base_model.weights)

# weight에 가중치 안받겠다고 해서 다 0 나오는거

[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
array([[[[-0.05981333,  0.09620547, -0.0222936 , ..., -0.07066211,
           0.06463021,  0.05319995],
         [ 0.03192008,  0.05130023,  0.05740128, ...,  0.043532  ,
           0.04389484, -0.05022942],
         [-0.02960535,  0.02282721,  0.01139711, ..., -0.09435022,
          -0.05883463, -0.02308311]],

        [[-0.04563003,  0.00043482,  0.01546253, ..., -0.03013188,
          -0.07155429,  0.07321198],
         [-0.06640771,  0.06563132,  0.08027969, ..., -0.06146029,
          -0.07148142,  0.01924549],
         [ 0.09805591,  0.08526534,  0.07042979, ...,  0.05483466,
           0.00328367, -0.09868664]],

        [[-0.02898922,  0.05280124, -0.08751019, ..., -0.07829888,
          -0.09464845,  0.01627842],
         [ 0.0537063 ,  0.01623321,  0.02052361, ..., -0.0004434 ,
          -0.0236409 ,  0.02945258],
         [ 0.0200443 ,  0.0186261 ,  0.00842627, ...,  0.07721075,
           0.0493

In [5]:
### 기본 모델의 각 계층(layer) 별 가중치 확인하기

# 생성된 모델의 전체 계층(layer) 확인
layers = base_model.layers
print(f'생성된 모델의 전체 계층 확인 : \n{layers}')

print('-'*88)

# 계층별 가중치 확인
for layer in layers:
    print(f'name = {layer.name}')
    print(f'weights : \n{layer.weights}')
    print('='*88)

생성된 모델의 전체 계층 확인 : 
[<keras.src.engine.input_layer.InputLayer object at 0x00000147F1ED65D0>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F21BFED0>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F194B450>, <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x00000147F2022910>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F2170450>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F22792D0>, <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x00000147F23D65D0>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F2294AD0>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F233E350>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F23D7D90>, <keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x00000147F23D56D0>, <keras.src.layers.convolutional.conv2d.Conv2D object at 0x00000147F2211750>, <keras.src.layers.convolutional.conv2d.

In [6]:
### 기본 모델의 각 계층 인덱스와 이름 확인 (가중치는 너무 기니까 패스)

 # enumerate() 함수 --> index 생성  / (enumerate = 인덱스 자동 생성해주는 기능)
for idx,layer in enumerate(base_model.layers):
    print(f'인덱스 = {idx}, 이름 = {layer.name}')   # name 자리에 weights를 넣으면 가중치 확인 가능

인덱스 = 0, 이름 = input_2
인덱스 = 1, 이름 = block1_conv1
인덱스 = 2, 이름 = block1_conv2
인덱스 = 3, 이름 = block1_pool
인덱스 = 4, 이름 = block2_conv1
인덱스 = 5, 이름 = block2_conv2
인덱스 = 6, 이름 = block2_pool
인덱스 = 7, 이름 = block3_conv1
인덱스 = 8, 이름 = block3_conv2
인덱스 = 9, 이름 = block3_conv3
인덱스 = 10, 이름 = block3_pool
인덱스 = 11, 이름 = block4_conv1
인덱스 = 12, 이름 = block4_conv2
인덱스 = 13, 이름 = block4_conv3
인덱스 = 14, 이름 = block4_pool
인덱스 = 15, 이름 = block5_conv1
인덱스 = 16, 이름 = block5_conv2
인덱스 = 17, 이름 = block5_conv3
인덱스 = 18, 이름 = block5_pool
인덱스 = 19, 이름 = flatten
인덱스 = 20, 이름 = fc1
인덱스 = 21, 이름 = fc2
인덱스 = 22, 이름 = predictions


In [7]:
### 출력층 수정 --> 재구성 모델 생성

'''
### 기존 모델의 출력 units(정답 레이블)의 수 = 1000개
### 우리에게 필요한 출력 units=2 --> 강아지 / 고양이
### 출력층 수정 --> 재구성 모델 생성
'''

# 출력 units 설정
units = 2

# 출력층 수정
x = base_model.get_layer(index=21).output
predictions = tf.keras.layers.Dense(units=units, activation='softmax')(x)#이거자체가 함수 x

###### f(x)

# 21번째의 출력이 있어야 22번째 재정의하는 곳에 데이터를 집어넣어서 데이터값을 끌어낼 수 있음.
# 

### 재구성 모델 생성

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# 결과 확인

print(f'재구성한 모델의 구조')
model.summary()



'''
tf.keras.Sequential()과

tf.keras.Model()는 매우 다르다 
차이점 복습하기 
'''



재구성한 모델의 구조
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128

'\ntf.keras.Sequential()과\n\ntf.keras.Model()는 매우 다르다 \n차이점 복습하기 \n'

In [8]:
base_model.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [9]:
base_model.get_layer(index=0).output

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>

In [10]:
## 위에 출력층 둘다 같은 식.

## 데이터 증식

In [11]:
## zip 파일 압축 해제

!unzip 'D:\Code\DataSets/cats_and_dogs.zip' -d 'D:\Code\DataSets'


'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [12]:
### 이미지의 수 확인
import os
# 이미지 폴더 경로 설정
train_dir = 'D:\Code\DataSets\cats_and_dogs/train'
val_dir = 'D:\Code\DataSets\cats_and_dogs/val'
test_dir = 'D:\Code\DataSets\cats_and_dogs/test'

# 학습용 폴더에 들어있는 강아지 이미지의 수 확인
dogs_list = os.listdir(train_dir + '/' + 'dogs')
# dogs_list = os.listdir(os.path.join(train_dir, 'dogs'))
print(f'학습용 dogs 폴더에 들어있는 강아지 이미지 파일의 이름 : \n{dogs_list}')

print('*'*80)

print(f'학습용 dogs 폴더에 들어있는 강아지 이미지의 수 = {len(dogs_list)}개')

# os는 이미지처리할 때 많이 쓰임.

print('*'*80)

# 검증용 폴더에 들어있는 고양이 이미지의 수 확인
cats_list = os.listdir(train_dir + '/' + 'cats')

print(f'학습용 cats 폴더에 들어있는 고양이 이미지 파일의 이름 : \n{cats_list}')

print('*'*80)

print(f'학습용 cats 폴더에 들어있는 고양이 이미지의 수 = {len(cats_list)}개')


print('*'*80)


학습용 dogs 폴더에 들어있는 강아지 이미지 파일의 이름 : 
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg', 'dog.106.jpg', 'dog.107.jpg', 'dog.108.jpg', 'dog.109.jpg', 'dog.11.jpg', 'dog.110.jpg', 'dog.111.jpg', 'dog.112.jpg', 'dog.113.jpg', 'dog.114.jpg', 'dog.115.jpg', 'dog.116.jpg', 'dog.117.jpg', 'dog.118.jpg', 'dog.119.jpg', 'dog.12.jpg', 'dog.120.jpg', 'dog.121.jpg', 'dog.122.jpg', 'dog.123.jpg', 'dog.124.jpg', 'dog.125.jpg', 'dog.126.jpg', 'dog.127.jpg', 'dog.128.jpg', 'dog.129.jpg', 'dog.13.jpg', 'dog.130.jpg', 'dog.131.jpg', 'dog.132.jpg', 'dog.133.jpg', 'dog.134.jpg', 'dog.135.jpg', 'dog.136.jpg', 'dog.137.jpg', 'dog.138.jpg', 'dog.139.jpg', 'dog.14.jpg', 'dog.140.jpg', 'dog.141.jpg', 'dog.142.jpg', 'dog.143.jpg', 'dog.144.jpg', 'dog.145.jpg', 'dog.146.jpg', 'dog.147.jpg', 'dog.148.jpg', 'dog.149.jpg', 'dog.15.jpg', 'dog.150.jpg', 'dog.151.jpg', 'dog.152.jpg', 'dog.153.jpg', 'dog.154.jpg', 'dog.155.jpg', 'dog.156.jpg'

# 데이터 증식(augmentation)

In [13]:
## 데이터 증식의 의미와 방법
#- 의미 : 원본 이미지에 인위적인 변화를 주어 학습 데이터의 양을 늘린다. 과대적합 방지
#- 증식의 방법
#1. 회전(rotation)
#2. 이동(shift)
#3. 밀림(shearing)
#4. 확대/축소(zoom)
#5. 상하좌우 반전(flip) 등

In [14]:
## tf.keras에서 제공하는 데이터 증식과 관련된 유용한 함수들
#
#- ImageDataGenerator() : 하이퍼파라미터 설정을 통해서 데이터 증식 방식에 대해서 설정
#train_datagen = ImageDataGenerator(하이퍼파라미터)
#val_datagen = ImageDataGenerator ... 이후 pdf 참조

In [15]:
### ImageDataGenerator() 함수 --> 데이터 증식 조건 설정

# 이미지 크기 설정

HEIGHT=224
WIDTH=224


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    
)

# 검증용 데이터에 대한 증식 조건 설정 --> MinMaxScaling 적용

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
)





In [16]:
### 이미지 증식 test

# 증식된 이미지 저장 경로 설정

save_path = 'D:\Code\Deep Learning/augmentation'

for name in dogs_list[:10]:
    img = Image.open(train_dir + '/' + 'dogs' + '/' + name)
    img = np.array(img)
    img = np.expand_dims(img, axis=0)
    count=0
    for data in train_datagen.flow(img, batch_size=1, save_to_dir=save_path, save_prefix='dog', save_format='jpg'):
        count+=1
        # 이미지 당 5개씩 증식된 이미지 생성
        if count>4:
            break

In [40]:
### ImageDataGenerator() 함수와 flow_from_directory() 함수를 사용하여 학습 데이터 증식

# 학습 데이터 증식

train_generator = train_datagen.flow_from_directory( # 학습용에느 2천개가 이미지가 있고, 그래서 클래스가 2개다
    directory=train_dir,
    batch_size=60, 
    target_size=(HEIGHT, WIDTH),
    shuffle=True,
    class_mode='categorical'
)

# 검증 데이터 증식(내용 : 증식X. MinMaxScaling 실행) --> 전처리 후 데이터 공급


val_generator = val_datagen.flow_from_directory( # 검증용에는 980개의 이미지가 있고, 클래스가 2개다.
    directory=val_dir,
    batch_size=60, 
    target_size=(HEIGHT,WIDTH),
    shuffle=True,
    class_mode='categorical'
)

# 검증용 데이터는 절대 손대지 않음. 전처리만 하는 것임.
# 형식적으로는 돼있으나 내용은 증식X, 전처리만 MinMaxScaling만 한다
    
    
    
# class_mode가 중요함!!!###########################

# train_generator에는 설정해준 batch_size만큼의 증식된 이미지 + 그 이미지의 정답레이블이 공급이 됨.
# ==> X_train
# Onehot 으로 들어가던 Label로 들어가던 얘가 y_train 역할을 수행함.

Found 2000 images belonging to 2 classes.


Found 980 images belonging to 2 classes.


In [42]:
print(val_dir)

D:\Code\DataSets\cats_and_dogs/val


In [43]:
#확인용 
# ### 생성되는 데이터와 정답 레이블 확인

# 결과물 생성 : train_generator --> images, labels
image, labels = next(train_generator) #X_train, y_train

# 정답 레이블 확인
print(labels)

print('*'*80)

# 생성된 이미지 배열 확인
print(image.shape)

[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
********************************************************************************
(60, 224, 224, 3)


### Callbacks

In [44]:
#tensorflow.keras.callbacks
#- 딥러닝 학습 시 종료 조건이나 모델 저장 조건 등을 사용자가 원하는 대로 설정할 수 있도록
#클래스와 메서드를 제공하는 모듈
#############-  EarlyStopping 클래스
#1 . 학습의 상태를 더 이상 개선시키지 못할 경우 학습을 강제로 종료시킬 수 있는 기능을 제공
'''2 . 사용방법
#early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
#                                              mode='min', 손실을 기준으로 한다면 최소가 될때를 기준으로 보겠다. 
#                                              verbose=1, 1로 설정하면 에폭당 학습이 진행되는 상황에 대해서 보고를 받을 수 있다. 귀찮으면 0으로 설정.
#                                              patience=5 몇번 참아줄지를 정하라.
#)
#
#3 . 매개 변수
#monitor : 학습 상태 판단의 기준, 'val_loss'을 입력할 경우 val_loss가 더 이상 감소하지 않을 경우에
#EarlyStopping 적용
#
#verbose : 학습의 진행 사항 표시 여부 결정, 학습의 진행 사항을 표시할 경우 verbose=1
#patience : 학습 상태가 개선 되지 않을 경우 참아주는 횟수

'''

"2 . 사용방법\n#early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', \n#                                              mode='min', 손실을 기준으로 한다면 최소가 될때를 기준으로 보겠다. \n#                                              verbose=1, 1로 설정하면 에폭당 학습이 진행되는 상황에 대해서 보고를 받을 수 있다. 귀찮으면 0으로 설정.\n#                                              patience=5 몇번 참아줄지를 정하라.\n#)\n#\n#3 . 매개 변수\n#monitor : 학습 상태 판단의 기준, 'val_loss'을 입력할 경우 val_loss가 더 이상 감소하지 않을 경우에\n#EarlyStopping 적용\n#\n#verbose : 학습의 진행 사항 표시 여부 결정, 학습의 진행 사항을 표시할 경우 verbose=1\n#patience : 학습 상태가 개선 되지 않을 경우 참아주는 횟수\n\n"

In [45]:
################# ModelCheckpoint 클래스
#1 학습된 모델을 저장할 때 저장 조건을 사용자가 원하는 대로 설정할 수 있는 기능 제공
#2 사용방법

#checkpoint = tf.keras.callbacks.ModelCheckpoint(
#    filepath = checkpoint_dr + '/' _ file_name,
#    monitor = 'val_loss',
#    mode = 'min',
#    verbose = 1,
#    save_best_only=True) # 반드시 해줘야한다. 가장 성능이 좋은 모델만 저장하게 된다.
#'''
'''
3 매개변수
checkpoint_path : 모델을 저장할 경로 입력
monitor : 모델을 저장할 때의 기준, 'val_loss'을 입력할 경우 validation set의 val_loss가 가장 낮을 때 저장
save_best_only : True인 경우, monitor 되고 있는 값을 기준으로 가장 좋은 값으로 모델이 저장, False인 경우,
매 epoch마다 모델 저장 (model0, model1, model2....)

'''

"\n3 매개변수\ncheckpoint_path : 모델을 저장할 경로 입력\nmonitor : 모델을 저장할 때의 기준, 'val_loss'을 입력할 경우 validation set의 val_loss가 가장 낮을 때 저장\nsave_best_only : True인 경우, monitor 되고 있는 값을 기준으로 가장 좋은 값으로 모델이 저장, False인 경우,\n매 epoch마다 모델 저장 (model0, model1, model2....)\n\n"

## 모델 컴파일 및 학습

In [46]:
### 학습 결과를 저장할 디렉토리 생성

save_path = 'D:\Code\DataSets\cats_and_dogs/base_weights.hdf5'

# fine-tuning은 사전학습된 내용을 받아서 한번 더 학습을 시킨 것이다.

# 과대 적합을 방지하기 위한 조기 종료 설정
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='min',
    verbose=1,
    patience=3 #학습의 횟수가 적을때는 2,3번으로 줄여줘야 조기종료의 의미가 개선이 됨. 따라서 보통 학습횟수가 클 때 적용한다
)

# 최적의 학습 결과를 저지하기 위한 ModelCheckpoint 설정

checkpoint = tf.keras.callbacks.ModelCheckpoint( #계속 best만 저장, 덮어씌우기하는것 개선될때만.. 기존 것을 업데이트할 것. 따라서 patience가 필요없다
    filepath=save_path,
    monitor='val_accuracy',
    mode = 'min',
    verbose=1, #저장이 된다 안된다 표시하고 싶을 때
    save_best_only=True, 
)
    # 컴파일 설정 필요
    # 클래스모드가 컴파일에 영향을 준다.
# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy', #손실화함수 
              optimizer=optimizer, #가중치최적화  
              metrics = ['accuracy'] #가중치 평가지표, 검증용데이터에 대한 검증용. 정확도로 검증하겠다.
)
#################### Early Stop과 Checkpoint는 결국 옵션이다. 컴파일은 단독으로 중요하다!!

# 학습 진행
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)

# 데이터가 너무 많고 안된다.... 사전학습된걸 파인튜닝(fine-tuning)해보자.

# 충분하게 큰 데이터로 학습된, 그 데이터를 맞춤형으로 만들자.

 8/34 [======>.......................] - ETA: 16:21 - loss: 0.6929 - accuracy: 0.5292

## 학습 결과 시각화

In [26]:
# accuracy, loss 추출 --> 그래프의 y축 설정

# print(history.history)

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# 그래프 축
epochs = np.arange(1, len(train_acc)+1)

# accuracy 시각화
plt.plot(epochs, train_acc, 'b', label ='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.show()
#성능이 개선안된, 학습이 안된 것의 시각화

print('*'*80)

# loss 시각화
plt.plot(epochs, train_loss, 'r', label ='Training acc')
plt.plot(epochs, val_loss, 'm', label='Validation acc')
plt.title('Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

# 데이터 증식 + Fine-Tuning을 이용하여 이미지 분류하기

## Model Generation

In [30]:
### 기본 모델 생성
pretrained_model = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3))

58889256/58889256 [==============================] - 6s 0us/step


In [31]:
######################## pretrained_model의 모든 계층 확인  --> 동결(freezing)
for layer in pretrained_model.layers:
    layer.trainable=False

In [32]:
### pretrained_model 모든 계층 동결 후 모델 구조 확인
pretrained_model.summary() #구조가 바뀐다.

# 여기에 덴스를 붙일 것. 재구성 모델을 만들기 위해서!

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
### 미세조정(fine-tuning) 모델 정의

# 출력 units 설정 (강아지,고양이 )
units=2

# 모델 생성
finetuned_model = tf.keras.Sequential()
finetuned_model.add(pretrained_model)
finetuned_model.add(tf.keras.layers.Flatten())
finetuned_model.add(tf.keras.layers.Dense(units=256, activation='relu'))
finetuned_model.add(tf.keras.layers.Dropout()) # 입력의 총합을 구할 때 연산량이 너무 많으니까 이중에 랜덤하게 몇개만 샘플링해서 입력의 총합을 만들것이다.
# 계산을 대충하겠다!!!
finetuned_model.add(tf.keras.layers.Dense(units=units, activation='softmax'))

# 모델 구조 확인
finetuned_model.summary()




Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [36]:
# Sequential()함수의 장단점과 model.add() 의 장단점을 복습할 것!

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")
